In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## Importing the data the data

In [ ]:
housing = pd.read_csv("/home/jupyter/hands-on-ml/data/housing.csv")

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5,3.0, 4.5, 6., np.inf],
                               labels=[1,2,3,4,5])

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_data = housing.loc[train_index]
    strat_test_data = housing.loc[test_index]

In [ ]:
for set_ in (strat_train_data, strat_test_data):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
housing = strat_train_data.drop("median_house_value", axis=1)
housing_labels = strat_train_data["median_house_value"].copy()

## Data Cleaning

Most of ML algorithms cannot deal with missing features. Scikit-Learn provides a handy class to take care of the missing values: SimpleImputer.

Remember that missing inputations should be done using training dataset, we can not use data from the validation/test dataset.

In [ ]:
imputer = SimpleImputer(strategy="median")

Since the median can only be computed on numerical attributes, you need to create a copy of the data without the text attribute ocean_proximity

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)

In [ ]:
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
housing_num.median().values

Now, we can use the "trained" imputer to transform the training set by replacing missing values with the learned medians

In [ ]:
X = imputer.transform(housing_num)

The results is a plain NumPy array containing the transformed features. If we want to put it back into a pandas DataFrame, it is simple

In [ ]:
housing_tr = pd.DataFrame(X,
                          columns=housing_num.columns,
                          index=housing_num.index)

## Handling Text and Categorical Attributes

Let's convert the categorical variables from text to number

In [ ]:
housing_cat = housing[["ocean_proximity"]]
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
ordinal_encoder.categories_

One issue with this representation is that ML algorithms will assume that two nearby values are more similiar than two distant values.
Let's use one-hot encoding

In [ ]:
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
housing_cat_1hot.toarray()
cat_encoder.categories_

## Custom Transformers

Although Scikit-Learn provides many useful transformers, we will need to write our own code for tasks such as custom cleanup operations or combining specific attributes. 
Here, it is a small transformer class that adds the combined attributed we discussed earlier.

In [ ]:
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix]/X[:, households_ix]
        population_per_household = X[:, population_ix]/X[:, households_ix]
        
        if self.add_bedrooms_per_room:
            beedroms_per_room = X[:, bedrooms_ix]/X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, beedroms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [ ]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

## Transform Pipelines

In [ ]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("attribs_adder", CombinedAttributesAdder()),
    ("std_scaler", StandardScaler()),
])

In [ ]:
housing_num_tr = num_pipeline.fit_transform(housing_num)

So far, we have handled the categorical columns and the numerical columns separately. It would be more convenient to have a single transformer able to handle all columns, applying the appropiate trnsformations to columns

In [ ]:
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

In [ ]:
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs)
])

In [ ]:
housing_prepared = full_pipeline.fit_transform(housing)
print(housing_prepared.shape)
print(housing.shape)

The constructor requires a list of tuples, where each tuple contains a name, a transformer, and a list of names (or indices) of columns that the transformer should be applied to.

Note that OneHotEncoder returns a sparse matrix,while the num_pipeline returns a dense matrix. When there is such a mix of sparse and dense matrix, the ColumnTransformer estimates the density of the final matrix. It returns a sparse matrix if the density is lower than a given threshold. In this example, it returns a dense matrix.